In [1]:
import pandas as pd
import numpy as np
from GPSPhoto import gpsphoto
import sys
from math import radians, cos, sin, asin, sqrt, pi
import urllib.request
import folium
from  pop_up_function import map_pop_up,location_map_path
import json
import os
import time
from exif_function import exif_file_df
from get_van_amenities import read_van_data
from pykalman import KalmanFilter
from get_distance_smoothed import distance,smooth
from get_transport_nearby import transportation, nearby_locations

In [2]:
osm_data=read_van_data("amenities-vancouver.json")
osm_data = osm_data.drop(columns='index')

In [3]:
#amenity_type=osm_data.groupby(['amenity'],as_index=False)['amenity'].agg({'cnt':'count'})


In [4]:
df_w=osm_data[(osm_data['amenity']=='bench')|
            (osm_data['amenity']=='bus_station')|
            (osm_data['amenity']=='drinking_water')|
            (osm_data['amenity']=='driving_school')|
            (osm_data['amenity']=='library')|
            (osm_data['amenity']=='lounge')].reset_index().drop(columns='index')

df_b=osm_data[(osm_data['amenity']=='bicycle_parking')|
            (osm_data['amenity']=='bicycle_rental')|
            (osm_data['amenity']=='bicycle_repair_station')].reset_index().drop(columns='index')


df_d=osm_data[(osm_data['amenity']=='EVSE')|
            (osm_data['amenity']=='car_rental')|
            (osm_data['amenity']=='car_rep')|
            (osm_data['amenity']=='car_sharing')|
            (osm_data['amenity']=='car_wash')|
            (osm_data['amenity']=='charging_station')|
            (osm_data['amenity']=='fuel')|
            (osm_data['amenity']=='motorcycle_parking')|
            (osm_data['amenity']=='motorcycle_rental')|
            (osm_data['amenity']=='parking')|
            (osm_data['amenity']=='parking_entrance')|
            (osm_data['amenity']=='parking_space')].reset_index().drop(columns='index')
   

In [5]:
exif_file_df("photos")
df = pd.read_csv("image_locations.csv")
df = df.drop(columns='Unnamed: 0')
df = df.sort_values(by=['time'])

In [6]:
df['time'] =  pd.to_datetime(df['time'], format='%Y:%m:%d %H:%M:%S')

In [7]:

if (df['time'].mean().hour<20)&(df['time'].mean().hour>5):
    df_intersting=osm_data[(osm_data['amenity']=='bbq')|
                           (osm_data['amenity']=='cafe')|
                           (osm_data['amenity']=='fast_food')|
                           (osm_data['amenity']=='food_court')|
                           (osm_data['amenity']=='ice_cream')|
                           (osm_data['amenity']=='juice_bar')|
                           (osm_data['amenity']=='restaurant')  ].reset_index().drop(columns='index')
    new_data1=nearby_locations(df_intersting,df)
else:
    df_intersting=osm_data[(osm_data['amenity']=='bar')|
                           (osm_data['amenity']=='biergarten')|
                           (osm_data['amenity']=='bistro')|
                           (osm_data['amenity']=='cinema')|
                           (osm_data['amenity']=='gambling')|
                           (osm_data['amenity']=='internet_cafe')|
                           (osm_data['amenity']=='nightclub')|
                           (osm_data['amenity']=='stripclub')|
                           (osm_data['amenity']=='theatre')].reset_index().drop(columns='index')
    new_data1=nearby_locations(df_intersting,df)


In [8]:
if (transportation(df)=="walking"):
    new_data=nearby_locations(df_w,df)

elif (transportation(df)=="biking"):
    new_data=nearby_locations(df_b,df)

else:
    new_data=nearby_locations(df_d,df)

new_data = new_data.drop(columns='index')
#new_data.to_csv('nerby_location_locations.csv')


In [9]:
print('Unfiltered distance: %0.2f' % (distance(df),))
smoothed_points = smooth(df[['lat','lon']])
print('Filtered distance: %0.2f' % (distance(smoothed_points),))


Unfiltered distance: 3468.06
Filtered distance: 2848.83


In [10]:

m = folium.Map(location=[49.246292,-123.116226],zoom_start=12)
fg=location_map_path(smoothed_points)
m.add_child(fg)


fg1 = folium.FeatureGroup(name='my map')
fg1=map_pop_up(new_data,fg1)
fg1=map_pop_up(new_data1,fg1)


m.add_child(fg1)   

In [11]:

m.save('output.html')
